In [1]:
# Import libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import keras
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
df_for_training = pd.read_csv("/content/drive/MyDrive/Datasets/Project/df_for_training.csv")

In [3]:
df_for_training.shape

(24548, 57)

In [4]:
# Function to create sequences
def singleStepSampler(df, window):
	xRes = []
	yRes = []
	for i in range(0, len(df) - window):
		res = []
		for j in range(0, window):
			r = []
			for col in df.columns:
				r.append(df[col][i + j])
			res.append(r)
		xRes.append(res)
		yRes.append(df[['Open', 'Close']].iloc[i + window].values)
	return np.array(xRes), np.array(yRes)

In [5]:
# Dataset splitting
SPLIT = 1.0
(xVal, yVal) = singleStepSampler(df_for_training, 20)
X_train = xVal[:int(SPLIT * len(xVal))]
y_train = yVal[:int(SPLIT * len(yVal))]
X_test = xVal[int(SPLIT * len(xVal)):]
y_test = yVal[int(SPLIT * len(yVal)):]
print(xVal.shape)
print(yVal.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24528, 20, 57)
(24528, 2)
(24528, 20, 57)
(24528, 2)
(0, 20, 57)
(0, 2)


In [6]:
# Building the model
multivariate_lstm = keras.Sequential()
multivariate_lstm.add(keras.layers.LSTM(200, input_shape=(X_train.shape[1], X_train.shape[2])))
multivariate_lstm.add(keras.layers.Dropout(0.2))
multivariate_lstm.add(keras.layers.Dense(2, activation='linear'))
multivariate_lstm.compile(loss = 'MeanSquaredError', metrics=['MAE'], optimizer='Adam')
multivariate_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 200)               206400    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 2)                 402       
                                                                 
Total params: 206802 (807.82 KB)
Trainable params: 206802 (807.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Fitting the data to the model
history = multivariate_lstm.fit(X_train, y_train, epochs=100)

Epoch 1/100
767/767 [==============================] - 9s 6ms/step - loss: 0.0054 - MAE: 0.0470
Epoch 2/100
767/767 [==============================] - 4s 5ms/step - loss: 0.0027 - MAE: 0.0329
Epoch 3/100
767/767 [==============================] - 4s 5ms/step - loss: 0.0023 - MAE: 0.0286
Epoch 4/100
767/767 [==============================] - 4s 5ms/step - loss: 0.0022 - MAE: 0.0271
Epoch 5/100
767/767 [==============================] - 5s 6ms/step - loss: 0.0020 - MAE: 0.0255
Epoch 6/100
767/767 [==============================] - 4s 5ms/step - loss: 0.0020 - MAE: 0.0252
Epoch 7/100
767/767 [==============================] - 4s 5ms/step - loss: 0.0020 - MAE: 0.0241
Epoch 8/100
767/767 [==============================] - 5s 6ms/step - loss: 0.0018 - MAE: 0.0238
Epoch 9/100
767/767 [==============================] - 4s 6ms/step - loss: 0.0018 - MAE: 0.0235
Epoch 10/100
767/767 [==============================] - 4s 5ms/step - loss: 0.0017 - MAE: 0.0228
Epoch 11/100
767/767 [=================

In [36]:
# Save the model to a specific path
model_path = '/content/drive/MyDrive/Final Year Project/multivariate_lstm_model2.h5'
multivariate_lstm.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
